In [2]:
from __future__ import annotations

import operator as op
import re
from ast import literal_eval
from collections import Counter, defaultdict
from dataclasses import dataclass, field
from functools import cache, cmp_to_key, reduce
from heapq import heappop, heappush
from itertools import cycle, permutations, product
from math import gcd, inf, lcm, prod

import black
import jupyter_black
import z3
from more_itertools import chunked, flatten
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2022/{day}-example.txt" if example else f"2022/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [74]:
# Day 1: Calorie Counting
elves = data(1, ints, "\n\n")
elves = sorted((sum(calories) for calories in elves), reverse=True)
print(f"Part 1: {first(elves)}")  # 69912
print(f"Part 2: {sum(elves[:3])}")  # 208180

Part 1: 69912
Part 2: 208180


In [75]:
# Day 2: Rock Paper Scissors
points = {
    "A X": 3 + 1,
    "A Y": 6 + 2,
    "A Z": 0 + 3,
    "B X": 0 + 1,
    "B Y": 3 + 2,
    "B Z": 6 + 3,
    "C X": 6 + 1,
    "C Y": 0 + 2,
    "C Z": 3 + 3,
}
points2 = {
    "A X": 0 + 3,
    "A Y": 3 + 1,
    "A Z": 6 + 2,
    "B X": 0 + 1,
    "B Y": 3 + 2,
    "B Z": 6 + 3,
    "C X": 0 + 2,
    "C Y": 3 + 3,
    "C Z": 6 + 1,
}
moves = data(2)
print(f"Part 1: {sum(points[move] for move in moves)}")  # 11150
print(f"Part 2: {sum(points2[move] for move in moves)}")  # 8295

Part 1: 11150
Part 2: 8295


In [76]:
# Day 3: Rucksack Reorganization
def common_item(rucksack):
    a = set(rucksack[: len(rucksack) // 2])
    b = set(rucksack[len(rucksack) // 2 :])
    return first(a & b)


def priority(char: str):
    if char.islower():
        return ord(char) - ord("a") + 1
    return ord(char) - ord("A") + 27


def group_common(group):
    a, b, c = map(set, group)
    return first(a & b & c)


rucksacks = data(3)
print(f"Part 1: {sum(priority(common_item(rucksack)) for rucksack in rucksacks)}")
print(
    f"Part 2: {sum(priority(group_common(group)) for group in chunked(rucksacks, 3))}"
)

Part 1: 7428
Part 2: 2650


In [77]:
# Day 4: Camp Cleanup
def fully_contains(a_low, a_high, b_low, b_high):
    if a_low <= b_low <= b_high <= a_high:
        return True
    if b_low <= a_low <= a_high <= b_high:
        return True
    return False


def overlaps(a_low, a_high, b_low, b_high):
    if a_high < b_low:
        return False
    if b_high < a_low:
        return False
    return True


def positive_ints(text):
    return [int(x) for x in re.findall("\d+", text)]


ranges = data(4, positive_ints)
print(f"Part 1: {sum(fully_contains(*ids) for ids in ranges)}")  # 540
print(f"Part 2: {sum(overlaps(*ids) for ids in ranges)}")  # 872

Part 1: 540
Part 2: 872


In [78]:
# Day 5: Supply Stacks
def initial_stacks(stack_input):
    stack_input = stack_input.splitlines()
    number_of_stacks = int(max(stack_input[-1]))
    stacks = [[] for _ in range(number_of_stacks)]
    for line in stack_input:
        # Every 4th character is a crate
        for stack, crate in enumerate(line[1::4]):
            if crate.isupper():
                stacks[stack].append(crate)
    for stack in stacks:
        stack.reverse()
    return stacks


def move_crates(number_to_move, start, dest):
    for _ in range(number_to_move):
        crate = stacks[start - 1].pop()
        stacks[dest - 1].append(crate)


def move_multiple(number_to_move, start, dest):
    left_behind, crates_to_move = (
        stacks[start - 1][:-number_to_move],
        stacks[start - 1][-number_to_move:],
    )
    stacks[start - 1] = left_behind
    stacks[dest - 1] += crates_to_move


stack_input, moves = data(5, sep="\n\n")
moves = [ints(line) for line in moves.splitlines()]

stacks = initial_stacks(stack_input)
for move in moves:
    move_crates(*move)
print(f'Part 1: {"".join(stack[-1] for stack in stacks)}')  # TLFGBZHCN

stacks = initial_stacks(stack_input)
for move in moves:
    move_multiple(*move)
print(f'Part 2: {"".join(stack[-1] for stack in stacks)}')  # QRQFHFWCL

Part 1: TLFGBZHCN
Part 2: QRQFHFWCL


In [79]:
# Day 6: Tuning Trouble
def start_of_block(message, window=4):
    i = 0
    while len(set(message[i : i + window])) < window:
        i += 1
    return i + window


message = data(6)[0]
print(f"Part 1: {start_of_block(message)}")  # 1566
print(f"Part 2: {start_of_block(message, window=14)}")  # 2265

Part 1: 1566
Part 2: 2265


In [80]:
# Day 7: No Space Left On Device
@dataclass
class Node:
    name: str
    parent: Node = None
    size: int = 0
    children: list[Node] = field(default_factory=list)

    def total_size(self):
        "Size of Node and all children."
        return sum(child.total_size() for child in self.children) + self.size

    def dir_sizes(self):
        for child in self.children:
            if child.children:
                # child is a directory
                yield from child.dir_sizes()
                yield child.total_size()


def create_tree(lines):
    root = Node("/")
    current = root
    for line in lines:
        match line.split():
            case "$", "cd", "/":
                current = root
            case "$", "cd", "..":
                current = current.parent
            case "$", "cd", dir:
                current = first(
                    child for child in current.children if child.name == dir
                )
            case "$", "ls":
                pass
            case "dir", dir:
                current.children.append(Node(dir, current))
            case size, file:
                current.children.append(Node(file, current, int(size)))
    return root


lines = data(7)
root = create_tree(lines)
print(f"Part 1: {sum(size for size in root.dir_sizes() if size <= 100_000)}")  # 1243729

TOTAL_SPACE = 70000000
NEEDED = 30000000
min_delete = NEEDED - TOTAL_SPACE + root.total_size()
print(
    f"Part 2: {min(size for size in root.dir_sizes() if size >= min_delete)}"
)  # 4443914

Part 1: 1243729
Part 2: 4443914


In [81]:
# Day 8: Treetop Tree House
def tree_line(row, col, dr, dc):
    "Return all tree positions to the edge in direction dr, dc"
    row += dr
    col += dc
    while (row, col) in trees:
        yield row, col
        row += dr
        col += dc


def visible(row, col):
    return any(
        all(trees[(r, c)] < trees[row, col] for r, c in tree_line(row, col, *dir))
        for dir in DIRECTIONS
    )


def scenic_score(row, col):
    total_score = 1
    for dir in DIRECTIONS:
        score = 0
        for r, c in tree_line(row, col, *dir):
            score += 1
            if trees[(r, c)] >= trees[(row, col)]:
                break
        total_score *= max(
            score, 1
        )  # Edge trees will have score 0 in some direction(s), make sure they still count
    return total_score


DIRECTIONS = ((0, 1), (0, -1), (1, 0), (-1, 0))
lines = data(8)
trees = {
    (row, col): int(num)
    for row, line in enumerate(lines)
    for col, num in enumerate(line)
}

print(f"Part 1: {sum(visible(*tree) for tree in trees)}")  # 1820
print(f"Part 2: {max(scenic_score(*tree) for tree in trees)}")  # 385112

Part 1: 1820


KeyboardInterrupt: 

In [ ]:
# Day 9: Rope Bridge
def parse_line(line):
    move, steps = line.split()
    steps = int(steps)
    return move, steps


def touching(r1, c1, r2, c2):
    adjacent = {
        (r, c) for r, c in product(range(r1 - 1, r1 + 2), range(c1 - 1, c1 + 2))
    }
    return (r2, c2) in adjacent


def move_knot(row, col, knot_row, knot_col):
    if row > knot_row:
        knot_row += 1  # Down
    elif row < knot_row:
        knot_row -= 1  # Up
    if col > knot_col:
        knot_col += 1  # Right
    elif col < knot_col:
        knot_col -= 1  # Left
    return knot_row, knot_col


def move_rope(rope, moves):
    row, col = (0, 0)
    visited = {rope[-1]}
    for move, steps in moves:
        for _ in range(steps):
            row += MOVES[move][0]
            col += MOVES[move][1]
            prev = (row, col)  # Head
            new_rope = []
            for knot in rope:
                if not touching(*prev, *knot):
                    knot = move_knot(*prev, *knot)
                new_rope.append(knot)
                prev = knot
            rope = new_rope
            visited.add(rope[-1])  # Tail
    return len(visited)


MOVES = {"D": (1, 0), "U": (-1, 0), "L": (0, -1), "R": (0, 1)}  # dr, dc
moves = data(9, parse_line)

knots = [(0, 0)]
print(f"Part 1: {move_rope(knots, moves)}")  # 5695
knots = [(0, 0) for _ in range(9)]
print(f"Part 2: {move_rope(knots, moves)}")  # 2434

Part 1: 5695
Part 2: 2434


In [ ]:
# Day 10: Cathode-Ray Tube
lines = data(10, str.split)
xs = [0, 1]
for line in lines:
    xs.append(xs[-1])
    match line:
        case "addx", x:
            xs.append(xs[-1] + int(x))
cycles = range(20, 220 + 1, 40)
print(f"Part 1: {sum(cycle * xs[cycle] for cycle in cycles)}")  # 14340

print("Part 2:")  # PAPJCBHP
for pixel, sprite in enumerate(xs[1:-1]):
    if pixel % 40 == 0:
        print()
    print("⬜️⬛️"[(sprite - 1 <= pixel % 40 <= sprite + 1) * 2], end="")

Part 1: 14340
Part 2:

⬛⬛⬛⬜⬜⬜⬛⬛⬜⬜⬛⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬛⬛⬜⬜⬛⬛⬛⬜⬜⬛⬜⬜⬛⬜⬛⬛⬛⬜⬜
⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬛⬛⬜⬜⬛⬛⬛⬛⬜⬛⬜⬜⬛⬜
⬛⬛⬛⬜⬜⬛⬛⬛⬛⬜⬛⬛⬛⬜⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬛⬛⬜⬜
⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜⬛⬛⬜⬜⬜⬛⬛⬜⬜⬛⬛⬛⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜

In [ ]:
# Day 11: Monkey in the Middle
class Monkey:
    def __init__(self, chunk):
        _, items, operation, divisor, if_true, if_false = chunk
        self.items = ints(items)
        self.operation = operation.split(" = ")[-1]
        self.divisor = int(divisor.split()[-1])
        self.target_monkeys = [int(if_false[-1]), int(if_true[-1])]
        self.inspections = 0


def monkey_business(monkeys, rounds=20, part1=True):
    all_divisors = prod(m.divisor for m in monkeys)
    for _ in range(rounds):
        for monkey in monkeys:
            while monkey.items:
                monkey.inspections += 1
                old = monkey.items.pop(0)
                worry_level = eval(monkey.operation)
                worry_level %= all_divisors
                if part1:
                    worry_level //= 3
                target = monkey.target_monkeys[worry_level % monkey.divisor == 0]
                monkeys[target].items.append(worry_level)
    inspections = sorted(m.inspections for m in monkeys)
    return inspections[-1] * inspections[-2]


chunks = data(11, str.splitlines, sep="\n\n", example=False)
monkeys = [Monkey(chunk) for chunk in chunks]
print(f"Part 1: {monkey_business(monkeys)}")  # 61005
monkeys = [Monkey(chunk) for chunk in chunks]
print(f"Part 2: {monkey_business(monkeys, rounds=10_000, part1=False)}")  # 20567144694

Part 1: 61005
Part 2: 20567144694


In [ ]:
# Day 12: Hill Climbing Algorithm
def a_star(starts, goal):
    """
    A* search from all `start` states in `starts` to `goal`.
    Need to provide functions: `heuristic`, `cost`, and `moves`
    """
    frontier = []
    cost_so_far = defaultdict(lambda: inf)
    for start in starts:
        cost_so_far[start] = 0
        heappush(frontier, (0, start))

    while frontier:
        current = heappop(frontier)[1]
        if heuristic(current, goal) == 0:
            break
        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                heappush(frontier, (priority, next))

    return cost_so_far[goal]


def heuristic(current, goal) -> float:
    "Minimum estimation of cost to get from current to goal. Also used to determine when we have reached goal (heuristic returns 0)."
    # This implements cab distance from a to goal. State is (row, col) for a point.
    (r1, c1) = current
    (r2, c2) = goal
    return abs(r1 - r2) + abs(c1 - c2)


def cost(current, next):
    return 1


def moves(current):
    row, col = current
    candidates = [(row + 1, col), (row - 1, col), (row, col + 1), (row, col - 1)]
    return [pos for pos in candidates if pos in grid and grid[pos] <= grid[current] + 1]


def parse_lines(lines, start=(0, 0), goal=(0, 0)):
    grid = {}
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char == "S":
                start = (row, col)
                char = "a"
            if char == "E":
                goal = (row, col)
                char = "z"
            grid[(row, col)] = ord(char) - ord("a")
    return grid, start, goal


lines = data(12)
grid, start, goal = parse_lines(lines)

print(f"Part 1: {a_star([start], goal)}")  # 412
starts = [pos for pos in grid if grid[pos] == 0]
print(f"Part 2: {a_star(starts, goal)}")  # 402

Part 1: 412
Part 2: 402


In [ ]:
# Day 13: Distress signal
def parse_pairs(text):
    left, right = text.splitlines()
    left = literal_eval(left)
    right = literal_eval(right)
    return left, right


def correct_order(left, right):
    if isinstance(left, int) and isinstance(right, int):
        if left < right:
            return -1
        elif left > right:
            return 1
        else:
            return 0
    if isinstance(left, list) and isinstance(right, list):
        for left_, right_ in zip(left, right):
            if correct_order(left_, right_) != 0:
                return correct_order(left_, right_)
        return correct_order(len(left), len(right))
    if isinstance(left, int) and isinstance(right, list):
        return correct_order([left], right)
    if isinstance(left, list) and isinstance(right, int):
        return correct_order(left, [right])


def pairs_in_correct_order(pairs):
    result = 0
    for index, (left, right) in enumerate(pairs, 1):
        if correct_order(left, right) == -1:
            result += index
    return result


def decoder_key(pairs, first_divider=[[2]], second_divider=[[6]]):
    def packet_index(divider):
        return packets.index(divider) + 1

    packets = list(flatten(pairs))
    packets.extend([first_divider, second_divider])
    packets.sort(key=cmp_to_key(correct_order))
    return packet_index(first_divider) * packet_index(second_divider)


pairs = data(13, parse_pairs, sep="\n\n")
print(f"Part 1: {pairs_in_correct_order(pairs)}")  # 5555
print(f"Part 2: {decoder_key(pairs)}")  # 22852

Part 1: 5555
Part 2: 22852


In [ ]:
# Day 14: Regolith Reservoir
def get_rocks(coords):
    "Return a set with all points in straight lines between coords."
    rocks = set()
    prev_x, prev_y = 0, 0
    for (x, y) in chunked(coords, 2):
        if (prev_x, prev_y) == (0, 0):
            rocks.add((x, y))
        else:
            x_range = range(min(x, prev_x), max(x, prev_x) + 1)
            y_range = range(min(y, prev_y), max(y, prev_y) + 1)
            for x_, y_ in product(x_range, y_range):
                rocks.add((x_, y_))
        prev_x, prev_y = x, y
    return rocks


def next_grain_position(x, y):
    if (x, y + 1) not in grid:
        return x, y + 1  # Down
    elif (x - 1, y + 1) not in grid:
        return x - 1, y + 1  # Left diagonal
    elif (x + 1, y + 1) not in grid:
        return x + 1, y + 1  # Right diagonal
    else:
        return x, y  # Can't flow further, rest here


def add_grain(start, max_y, part2):
    "Add a grain of sand. Return True if possible to add more sand."
    current = start
    while current := next_grain_position(*current):
        if current == next_grain_position(*current):
            grid[current] = SAND
            return current != start
        if current[1] >= max_y + 1:
            if part2:
                # Fill one step below lowest rock line
                grid[current] = SAND
                return True
            return False


def fill(start=(500, 0), part2=False):
    max_y = max(grid, key=lambda coord: coord[1])[1]
    while add_grain(start, max_y, part2):
        pass
    return sum(1 for coord in grid if grid[coord] == SAND)


lines = data(14, ints)
ROCK, SAND = 1, 2
grid = {coord: ROCK for line in lines for coord in get_rocks(line)}
print(f"Part 1: {fill()}")  # 858
print(f"Part 2: {fill(part2=True)}")  # 26845

Part 1: 858
Part 2: 26845


In [86]:
# Day 15: Beacon Exclusion Zone
def coordinates(text):
    return tuple((x, y) for x, y in chunked(ints(text), 2))


def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def no_beacon_zone(sensor):
    "Return four coordinates (top, bottom, left, right) that enclose the area where there is only one beacon"
    radius = manhattan_distance(sensor, closest[sensor])
    x, y = sensor
    return [(x, y - radius), (x, y + radius), (x - radius, y), (x + radius, y)]


def no_beacon_line(sensor, row):
    top, bottom, left, right = no_beacon_zone(sensor)
    if top[1] <= row <= bottom[1]:
        # row is affected by this zone
        x, y = sensor
        radius = manhattan_distance(sensor, closest[sensor])
        affected_width = radius - abs(y - row)
        return x - affected_width, x + affected_width
    return None


def no_beacon_lines(row):
    return [
        no_beacon_line(sensor, row) for sensor in closest if no_beacon_line(sensor, row)
    ]


def create_segments(row):
    lines = sorted(no_beacon_lines(row))
    segments = [lines.pop(0)]
    for start, end in lines:
        # print(f"{segments=}")
        prev_start, prev_end = segments.pop()
        # print(f"{prev_end=} {start=}")
        if start <= prev_end:
            segments.append((prev_start, max(end, prev_end)))
        else:
            segments.append((prev_start, prev_end))
            segments.append((start, end))
    return segments


def positions_without_beacon(row=2000000):
    segments = create_segments(row)
    assert len(segments) == 1
    return (
        segments[0][1]
        - segments[0][0]
        + 1
        - sum(1 for beacon in set(closest.values()) if beacon[1] == row)
    )


def tuning_frequency(search_space=4_000_000):
    for row in range(search_space):
        if len(create_segments(row)) > 1:
            y = row
            x = create_segments(y)[0][1] + 1
            return x * 4000000 + y


coords = data(15, coordinates)
closest = {sensor: beacon for sensor, beacon in coords}
print(f"Part 1: {positions_without_beacon()}")  # 5832528
print(f"Part 2: {tuning_frequency()}")  # 13360899249595, 3m

Part 1: 5832528
Part 2: 13360899249595


In [178]:
# Day 16: Proboscidea Volcanium (Part 1)
def parse_lines(lines):
    moves = {}
    pressure = {}
    for line in lines:
        valve, rate, destinations = parse("Valve {} has flow rate={:d}; {}", line)
        destinations = destinations.split(", ")
        destinations[0] = destinations[0][-2:]
        moves[valve] = tuple(destinations)
        if rate > 0:
            pressure[valve] = rate
    return moves, pressure


@cache
def released_pressure(room="AA", open_valves=frozenset(), time_remaining=30):
    if time_remaining == 0:
        return 0
    current_flow = sum(pressure[valve] for valve in open_valves)
    other_rooms = max(
        released_pressure(r, open_valves, time_remaining - 1) + current_flow
        for r in moves[room]
    )
    if room not in open_valves and room in pressure:
        # Possible to open valve
        new_valves = open_valves | frozenset({room})
        this_room = (
            released_pressure(room, new_valves, time_remaining - 1) + current_flow
        )
        return max(other_rooms, this_room)
    return other_rooms


lines = data(16)
moves, pressure = parse_lines(lines)
print(f"Part 1: {released_pressure()}")  # 2124, 4s

Part 1: 2124


In [6]:
# Day 16: Proboscidea Volcanium (Part 2 WIP)
@cache
def released_pressure(room="AA", open_valves=frozenset(), time_remaining=30):
    if time_remaining == 0:
        return 0
    current_flow = sum(pressure[valve] for valve in open_valves)
    other_rooms = max(
        [
            released_pressure(r, open_valves, time_remaining - distance[(room, r)])
            + current_flow * distance[(room, r)]
            for r in pressure
            if distance[(room, r)] <= time_remaining
        ]
        + [0]
    )
    if room in set(pressure) - open_valves:
        # Possible to open valve
        this_room = (
            released_pressure(room, open_valves | {room}, time_remaining - 1)
            + current_flow
        )
        return max(other_rooms, this_room)
    return other_rooms


lines = data(16)
moves, pressure, distance = parse_lines(lines)

# Reduce search space by only including rooms with valves
# Use https://en.wikipedia.org/wiki/Floyd–Warshall_algorithm to find distances

for k, i, j in product(moves, moves, moves):
    distance[i, j] = min(distance[i, j], distance[i, k] + distance[k, j])

released_pressure(time_remaining=30)  # 26 == 1608

2124

In [181]:
# Day 16 part 2
def parse_lines(lines):
    moves = {}
    valves = {}
    distance = defaultdict(lambda: inf)
    for line in lines:
        valve, rate, destinations = parse("Valve {} has flow rate={:d}; {}", line)
        destinations = destinations.split(", ")
        destinations[0] = destinations[0][-2:]
        moves[valve] = tuple(destinations)
        for dest in destinations:
            distance[valve, dest] = 1
        if rate > 0:
            valves[valve] = rate
    return moves, valves, distance


# based on https://github.com/juanplopes/advent-of-code-2022/blob/main/day16.py
def visit(
    start="AA",
    time_remaining=30,
    open_valves=frozenset(),
    flow=0,
):
    global total_pressure
    total_pressure[open_valves] = max(total_pressure[open_valves], flow)
    for valve in valves:
        new_time_remaining = time_remaining - distance[start, valve] - 1
        if valve in open_valves or new_time_remaining <= 0:
            continue
        visit(
            valve,
            new_time_remaining,
            open_valves | {valve},
            flow + new_time_remaining * valves[valve],
        )


lines = data(16)
moves, valves, distance = parse_lines(lines)

# Reduce search space by only including rooms with valves
# Use https://en.wikipedia.org/wiki/Floyd–Warshall_algorithm to find distances

for k, i, j in product(moves, moves, moves):
    distance[i, j] = min(distance[i, j], distance[i, k] + distance[k, j])


total_pressure = defaultdict(int)
visit()
print(f"Part 1: {max(total_pressure.values())}")  # 2124
total_pressure.clear()
visit(time_remaining=26)
part2 = max(
    v1 + v2
    for k1, v1 in total_pressure.items()
    for k2, v2 in total_pressure.items()
    if k1.isdisjoint(k2)
)
print(f"Part 2: {part2}")  # 2775

Part 1: 2124
Part 2: 2775


In [17]:
# Day 17: Pyroclastic Flow

# Time to bring out the complex numbers? Let the imaginary part be x, real part is y. Rock
# bottom is at y = 0. Each rock starts with left edge two units away from
# left wall and three units above the highest rock
MINUS = (0 + 2j, 0 + 3j, 0 + 4j, 0 + 5j)
PLUS = (2 + 3j, 1 + 2j, 1 + 3j, 1 + 4j, 0 + 3j)
EDGE = (2 + 4j, 1 + 4j, 0 + 2j, 0 + 3j, 0 + 4j)
LINE = (3 + 2j, 2 + 2j, 1 + 2j, 0 + 2j)
BOX = (1 + 2j, 1 + 3j, 0 + 2j, 0 + 3j)
ROCKS_TO_DROP = [MINUS, PLUS, EDGE, LINE, BOX]
LEFT_WALL = -1
RIGHT_WALL = 7
ROCK_BOTTOM = 0


def push(rock, jet):
    direction = -1j if jet == "<" else 1j
    return tuple(coord + direction for coord in rock)


def drop(rock):
    return tuple(coord - 1 for coord in rock)


def starting_position(rocks, rock):
    bottom_edge = max(rocks, key=lambda x: x.real).real + 4
    return tuple(coord + bottom_edge for coord in rock)


def sideways_hit(rocks, rock):
    return (
        any(coord in rocks for coord in rock)
        or min(coord.imag for coord in rock) == LEFT_WALL
        or max(coord.imag for coord in rock) == RIGHT_WALL
    )


def downward_hit(rocks, rock):
    return (
        any(coord in rocks for coord in rock)
        or min(coord.real for coord in rock) == ROCK_BOTTOM
    )


def height_after_dropping_rocks(jets, number_of_rocks=2022):
    rock_to_drop = 0
    heights = []
    rocks = {ROCK_BOTTOM}
    rock = starting_position(rocks, ROCKS_TO_DROP[rock_to_drop])
    for jet in cycle(jets):
        new_position = push(rock, jet)
        if not sideways_hit(rocks, new_position):
            rock = new_position
            # print(f"Push {jet} to {rock}")
        new_position = drop(rock)
        if not downward_hit(rocks, new_position):
            rock = new_position
            # print(f"Drop to {rock}")
        else:
            # print(f"Resting at {rock}")
            rocks |= {part for part in rock}
            rock_to_drop += 1
            rock_to_drop %= len(ROCKS_TO_DROP)
            heights.append(int(max(coord.real for coord in rocks)))
            rock = starting_position(rocks, ROCKS_TO_DROP[rock_to_drop])
            number_of_rocks -= 1
            if number_of_rocks == 0:
                break
            # print(f"\nReleasing new rock {rock}, {number_of_rocks} left")
    return heights


jets = open("2022/17.txt").read().strip()
heights = height_after_dropping_rocks(jets)
print(f"Part 1: {heights[-1]}")  # 3109

# Part 2, what happens after 1000000000000 rocks dropped? Let's assume there is a repeated
# pattern here.
def print_heights():
    prev = 0
    for rocks, height in enumerate(heights):
        print(height - prev, end="")
        prev = height


heights = height_after_dropping_rocks(jets, number_of_rocks=4000)
# print_heights()
# Copy this output into an edit buffer and start marking a section from the end to find
# the pattern. It seems to repeat at a 1725 cycle starting at 351 for my input.

# For the example input the cycle is 35 long and starts at position 46, value increases
# with 53 per cycle

increase = heights[351 + 1725] - heights[351]
# The value increases with 2659 for each cycle. Double check with different values:
assert heights[1725 + 351 + 8] == 2659 + heights[351 + 8]

# Now we can define a closed form function to calculate the value
def get_value(rocks_launched, increase=2659, cycle=1725, base=351):
    offset = (rocks_launched - 1 - base) % cycle
    cycles = (rocks_launched - 1 - base) // cycle
    return increase * cycles + heights[base + offset]


assert get_value(4000) == heights[4000 - 1]  # Value after 4000 rocks
print(f"Part 2: {get_value(1000000000000)}")  # 1541449275365

Part 1: 3109
Part 2: 1541449275365


In [3]:
# Day 18: Boiling Boulders
def neighbors(x, y, z):
    return {
        (x - 1, y, z),
        (x + 1, y, z),
        (x, y - 1, z),
        (x, y + 1, z),
        (x, y, z - 1),
        (x, y, z + 1),
    }


def open_sides(x, y, z):
    "Number of sides open to air for a cube in droplets"
    assert (x, y, z) in droplets
    CUBE_SIDES = 6
    return CUBE_SIDES - sum(1 for n in neighbors(x, y, z) if n in droplets)


def adjacent_sides(x, y, z):
    "Number of sides adjacent to a cube for a position outside of droplets"
    assert (x, y, z) not in droplets
    return sum(1 for n in neighbors(x, y, z) if n in droplets)


def steam_cubes(lower=-1, upper=22):
    "Cubes outside of the lava droplet. Need to provide lower and upper bound."
    outside = set()
    frontier = [(-1, -1, -1)]
    while frontier:
        current = frontier.pop()
        for cube in neighbors(*current):
            if (
                cube not in outside
                and cube not in droplets
                and all(lower <= coord <= upper for coord in cube)
            ):
                frontier.append(cube)
        outside.add(current)
    return outside


def ints_tuple(text):
    return tuple(ints(text))


droplets = set(data(18, ints_tuple))
print(f"Part 1: {sum(open_sides(*cube) for cube in droplets)}")  # 4364
print(f"Part 2: {sum(adjacent_sides(*steam) for steam in steam_cubes())}")  # 2508

Part 1: 4364
Part 2: 2508


In [78]:
# Day 19: Not Enough Minerals

# State = time remaining, materials (ore, clay, obsidian), robots
# Blueprint 1 in example
@cache
def geodes(blueprint, minutes=24, materials=(0, 0, 0), robots=(1, 0, 0)):
    if minutes <= 1:
        return 0
    remaining_minutes = minutes - 1
    ore_cost, clay_cost, obs_cost, geode_cost = blueprint
    ore, clay, obs = materials
    ore_r, clay_r, obs_r = robots
    if ore >= geode_cost[0] and obs >= geode_cost[1]:
        # Geode robot possible, make it and don't explore other possibilities
        geode_materials = (
            ore + ore_r - geode_cost[0],
            clay + clay_r,
            obs + obs_r - geode_cost[1],
        )
        # Number of geodes we can produce with this robot == remaining time
        return remaining_minutes + geodes(
            blueprint, remaining_minutes, geode_materials, robots
        )
    if remaining_minutes <= 1:
        return 0  # Producing robots other than geode will not help this late

    # Throw away excess ore and clay.
    # Idea from https://github.com/jonathanpaulson/AdventOfCode/blob/master/2022/19.py
    max_ore_cost = max(ore_cost, clay_cost, obs_cost[0], geode_cost[0])
    if ore >= minutes * max_ore_cost - ore_r * remaining_minutes:
        ore = minutes * max_ore_cost - ore_r * remaining_minutes
    if clay >= minutes * obs_cost[1] - clay_r * remaining_minutes:
        clay = minutes * obs_cost[1] - clay_r * remaining_minutes

    produced_materials = (ore + ore_r, clay + clay_r, obs + obs_r)
    # No new robots
    options = [geodes(blueprint, remaining_minutes, produced_materials, robots)]
    if ore >= obs_cost[0] and clay >= obs_cost[1]:
        # Obsidian robot
        obs_materials = (
            ore + ore_r - obs_cost[0],
            clay + clay_r - obs_cost[1],
            obs + obs_r,
        )
        obs_robots = (ore_r, clay_r, obs_r + 1)
        options.append(geodes(blueprint, remaining_minutes, obs_materials, obs_robots))
    if remaining_minutes <= 2:
        return max(options)  # Only obsidian robots might help this late
    if ore >= ore_cost and ore_r < max_ore_cost:
        # New ore robot possible. We never need more than max ore cost
        ore_materials = (ore + ore_r - ore_cost, clay + clay_r, obs + obs_r)
        ore_robots = (ore_r + 1, clay_r, obs_r)
        options.append(geodes(blueprint, remaining_minutes, ore_materials, ore_robots))
    if ore >= clay_cost and clay_r < obs_cost[1]:
        # New clay robot possible. Don't build more than needed
        clay_materials = (ore + ore_r - clay_cost, clay + clay_r, obs + obs_r)
        clay_robots = (ore_r, clay_r + 1, obs_r)
        options.append(
            geodes(blueprint, remaining_minutes, clay_materials, clay_robots)
        )
    return max(options)


def quality_levels(blueprints, show_progress=False):
    max_geodes = []
    for bp, blueprint in enumerate(blueprints, 1):
        geodes.cache_clear()  # Manually clearing cache speeds up execution
        max_geodes.append(geodes(blueprint))
        if show_progress:
            print(f"{bp:2d}: {max_geodes[-1]:4d}")
    return sum(id * opens for id, opens in enumerate(max_geodes, 1))


def longer_time(blueprints):
    times = []
    for bp in blueprints[:3]:
        times.append(geodes(bp, minutes=32))
    return prod(times)


def blueprints(lines):
    blueprints = []
    for line in lines:
        _, ore, clay, obs1, obs2, geo1, geo2 = parse(
            "Blueprint {:d}: Each ore robot costs {:d} ore. Each clay robot costs {:d} ore. Each obsidian robot costs {:d} ore and {:d} clay. Each geode robot costs {:d} ore and {:d} obsidian.",
            line,
        )
        blueprints.append((ore, clay, (obs1, obs2), (geo1, geo2)))
    return blueprints


lines = data(19)
print(f"Part 1: {quality_levels(blueprints(lines))}")  # 2301
print(f"Part 2: {longer_time(blueprints(lines))}")  # 10336; ~23s for both parts

Part 1: 2301
Part 2: 10336


In [193]:
# Day 20: Grove Positioning System
def decrypt(lines, mixes=1, decryption_key=1):
    numbers = [int(number) * decryption_key for number in lines]
    positions = list(range(len(numbers)))

    for _ in range(mixes):
        for position, number in enumerate(numbers):
            current_location = positions.index(position)
            positions.pop(current_location)
            new_location = (current_location + number) % len(positions)
            if new_location == 0:
                positions.append(position)
            else:
                positions.insert(new_location, position)

    zero = positions.index(numbers.index(0))
    return sum(
        numbers[positions[(zero + pos) % len(numbers)]] for pos in [1000, 2000, 3000]
    )


lines = data(20)
numbers = [int(number) for number in lines]
print(f"Part 1: {decrypt(lines)}") # 10707
print(f"Part 2: {decrypt(lines, mixes=10, decryption_key=811589153)}") # 2488332343098

Part 1: 10707
Part 2: 2488332343098


In [3]:
# Day 21: Monkey Math
def fill_in(values):
    global formulas
    for monkey in formulas.copy():
        func, a, b = formulas[monkey]
        if a in values and b in values:
            values[monkey] = func(values[a], values[b])
            del formulas[monkey]
    return values


def parse_lines(lines):
    operations = {"+": op.add, "-": op.sub, "*": op.mul, "/": op.floordiv}
    values = {}
    formulas = {}
    for line in lines:
        if p := parse("{}: {:d}", line):
            values[p[0]] = p[1]
        elif p := parse("{}: {} {} {}", line):
            formulas[p[0]] = (operations[p[2]], p[1], p[3])
    return values, formulas


lines = data(21)
values, formulas = parse_lines(lines)

prev_values = 0
while prev_values != len(values):
    prev_values = len(values)
    values = fill_in(values)
print(f'Part 1: {values["root"]}')  # 80326079210554

Part 1: 80326079210554


In [5]:
# Day 21: Monkey Math using z3
def solve(solve_for, lines):
    s = z3.Solver()
    operations = {"+": op.add, "-": op.sub, "*": op.mul, "/": op.truediv}
    monkeys = {}
    for line in lines:
        if p := parse("{}: {:d}", line):
            monkey, value = p
            monkeys[monkey] = z3.Int(monkey)
            if monkey != "humn" or solve_for != "humn":
                s.add(monkeys[monkey] == value)
        elif p := parse("{}: {} {} {}", line):
            monkey, left, operand, right = p
            monkeys[monkey] = z3.Int(monkey)
            monkeys[left] = z3.Int(left)
            monkeys[right] = z3.Int(right)
            if solve_for == "humn" and monkey == "root":
                s.add(monkeys[left] == monkeys[right])
            else:
                s.add(
                    monkeys[monkey]
                    == operations[operand](monkeys[left], monkeys[right])
                )
    s.check()
    return s.model().eval(monkeys[solve_for])


print(f'Part 1: {solve("root", data(21))}')  # 80326079210554
print(f'Part 2: {solve("humn", data(21))}')  # 3617613952378

Part 1: 80326079210554
Part 2: 3617613952378


In [75]:
# Day 22: Monkey Map
@dataclass
class Position:
    row: int
    col: int
    dir: int  # 0, 1, 2, 3 = E, S, W, N
    board: dict

    movement = ((0, 1), (1, 0), (0, -1), (-1, 0))  # ESWN

    def left(self):
        self.dir -= 1
        self.dir %= len(self.movement)

    def right(self):
        self.dir += 1
        self.dir %= len(self.movement)

    def forward(self, steps):
        dr, dc = self.movement[self.dir]
        for _ in range(steps):
            self.row += dr
            self.col += dc
            if (self.row, self.col) not in self.board:
                new_row, new_col = self.row, self.col
                if self.dir == 0:
                    new_col = min(c for r, c in self.board.keys() if r == self.row)
                elif self.dir == 2:
                    new_col = max(c for r, c in self.board.keys() if r == self.row)
                elif self.dir == 1:
                    new_row = min(r for r, c in self.board.keys() if c == self.col)
                elif self.dir == 3:
                    new_row = max(r for r, c in self.board.keys() if c == self.col)
                if self.board[new_row, new_col] == "#":
                    self.row -= dr
                    self.col -= dc
                    break
                self.row, self.col = new_row, new_col
            if self.board[self.row, self.col] == "#":
                self.row -= dr
                self.col -= dc
                break

    def forward_on_cube(self, steps):
        for _ in range(steps):
            dr, dc = self.movement[self.dir]
            self.row += dr
            self.col += dc
            if (self.row, self.col) not in self.board:
                if self.dir == 0:
                    if self.col == 100 and self.row <= 99:
                        new_dir = 3
                        new_row = 49
                        new_col = self.row + 50
                    elif self.col == 100:
                        new_dir = 2
                        new_row = 149 - self.row
                        new_col = 149
                    elif self.col == 50:
                        new_dir = 3
                        new_row = 149
                        new_col = self.row - 100
                    elif self.col == 150:
                        new_dir = 2
                        new_row = 149 - self.row
                        new_col = 99
                elif self.dir == 1:
                    if self.row == 200:
                        new_dir = 1
                        new_row = 0
                        new_col = self.col + 100
                    elif self.row == 150:
                        new_dir = 2
                        new_row = 100 + self.col
                        new_col = 49
                    elif self.row == 50:
                        new_dir = 2
                        new_row = self.col - 50
                        new_col = 99
                elif self.dir == 2:
                    if self.col == 49 and self.row <= 49:
                        new_dir = 0
                        new_row = 149 - self.row
                        new_col = 0
                    elif self.col == 49:
                        new_dir = 1
                        new_row = 100
                        new_col = self.row - 50
                    elif self.row <= 149:
                        new_dir = 0
                        new_row = 149 - self.row
                        new_col = 50
                    elif self.row <= 199:
                        new_dir = 1
                        new_row = 0
                        new_col = self.row - 100
                elif self.dir == 3:
                    if self.row == -1 and self.col <= 99:
                        new_dir = 0
                        new_row = 100 + self.col
                        new_col = 0
                    elif self.row == -1 and self.col <= 199:
                        new_dir = 3
                        new_row = 199
                        new_col = self.col - 100
                    elif self.row == 99:
                        new_dir = 0
                        new_row = 50 + self.col
                        new_col = 50
                if self.board[new_row, new_col] == "#":
                    # print("Hit a wall wrapping around, move back")
                    self.row -= dr
                    self.col -= dc
                    break
                self.dir, self.row, self.col = new_dir, new_row, new_col
                # print(f"{self.row, self.col}")
            if self.board[self.row, self.col] == "#":
                # print(f"Hit wall at current face {self.row, self.col}")
                self.row -= dr
                self.col -= dc
                break

    def password(self):
        return (self.row + 1) * 1000 + (self.col + 1) * 4 + self.dir


def create_board(lines):
    board = {}
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char != " ":
                board[row, col] = char
    return board


def next_move(path):
    steps = ints(path)
    turns = re.findall("[RL]", path)
    for step, turn in zip(steps, turns):
        yield step
        yield turn
    yield steps[-1]


def password(board, path, cube=False):
    position = Position(0, min(board)[1], 0, board)
    for move in next_move(path):
        match move:
            case "R":
                position.right()
            case "L":
                position.left()
            case _:
                if cube:
                    position.forward_on_cube(move)
                else:
                    position.forward(move)
    return position.password()


lines, path = data(22, sep="\n\n")
lines = lines.splitlines()
print(f"Part 1: {password(create_board(lines), path)}")  # 197160
print(f"Part 2: {password(create_board(lines), path, cube=True)}")  # 145065

Part 1: 197160
Part 2: 145065


In [83]:
# Day 22: Monkey Map
@dataclass
class Position:
    row: int
    col: int
    dir: int
    board: dict

    movement = ((0, 1), (1, 0), (0, -1), (-1, 0))  # ESWN

    def left(self):
        self.dir -= 1
        self.dir %= len(self.movement)

    def right(self):
        self.dir += 1
        self.dir %= len(self.movement)

    def forward(self, steps):
        dr, dc = self.movement[self.dir]
        for _ in range(steps):
            self.row += dr
            self.col += dc
            if (self.row, self.col) not in self.board:
                new_row, new_col = self.row, self.col
                if self.dir == E:
                    new_col = min(c for r, c in self.board.keys() if r == self.row)
                elif self.dir == W:
                    new_col = max(c for r, c in self.board.keys() if r == self.row)
                elif self.dir == S:
                    new_row = min(r for r, c in self.board.keys() if c == self.col)
                elif self.dir == N:
                    new_row = max(r for r, c in self.board.keys() if c == self.col)
                if self.board[new_row, new_col] == "#":
                    self.row -= dr
                    self.col -= dc
                    break
                self.row, self.col = new_row, new_col
            if self.board[self.row, self.col] == "#":
                self.row -= dr
                self.col -= dc
                break

    def forward_on_cube(self, steps):
        for _ in range(steps):
            dr, dc = self.movement[self.dir]
            self.row += dr
            self.col += dc
            if (self.row, self.col) not in self.board:
                if self.dir == E:
                    if self.col == 100 and self.row <= 99:
                        new_dir, new_row, new_col = N, 49, self.row + 50
                    elif self.col == 100:
                        new_dir, new_row, new_col = W, 149 - self.row, 149
                    elif self.col == 50:
                        new_dir, new_row, new_col = N, 149, self.row - 100
                    elif self.col == 150:
                        new_dir, new_row, new_col = W, 149 - self.row, 99
                elif self.dir == W:
                    if self.col == 49 and self.row <= 49:
                        new_dir, new_row, new_col = E, 149 - self.row, 0
                    elif self.col == 49:
                        new_dir, new_row, new_col = S, 100, self.row - 50
                    elif self.row <= 149:
                        new_dir, new_row, new_col = E, 149 - self.row, 50
                    elif self.row <= 199:
                        new_dir, new_row, new_col = S, 0, self.row - 100
                elif self.dir == S:
                    if self.row == 200:
                        new_dir, new_row, new_col = S, 0, self.col + 100
                    elif self.row == 150:
                        new_dir, new_row, new_col = W, 100 + self.col, 49
                    elif self.row == 50:
                        new_dir, new_row, new_col = W, self.col - 50, 99
                elif self.dir == N:
                    if self.row == -1 and self.col <= 99:
                        new_dir, new_row, new_col = E, 100 + self.col, 0
                    elif self.row == -1 and self.col <= 199:
                        new_dir, new_row, new_col = N, 199, self.col - 100
                    elif self.row == 99:
                        new_dir, new_row, new_col = E, 50 + self.col, 50
                if self.board[new_row, new_col] == "#":
                    self.row -= dr
                    self.col -= dc
                    break
                self.dir, self.row, self.col = new_dir, new_row, new_col
            if self.board[self.row, self.col] == "#":
                self.row -= dr
                self.col -= dc
                break

    def password(self):
        return (self.row + 1) * 1000 + (self.col + 1) * 4 + self.dir


def create_board(lines):
    board = {}
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char != " ":
                board[row, col] = char
    return board


def next_move(path):
    steps = ints(path)
    turns = re.findall("[RL]", path)
    for step, turn in zip(steps, turns):
        yield step
        yield turn
    yield steps[-1]


def password(board, path, cube=False):
    position = Position(0, min(board)[1], 0, board)
    for move in next_move(path):
        match move:
            case "R":
                position.right()
            case "L":
                position.left()
            case _:
                if cube:
                    position.forward_on_cube(move)
                else:
                    position.forward(move)
    return position.password()


E, S, W, N = 0, 1, 2, 3

lines, path = data(22, sep="\n\n")
lines = lines.splitlines()
print(f"Part 1: {password(create_board(lines), path)}")  # 197160
print(f"Part 2: {password(create_board(lines), path, cube=True)}")  # 145065

Part 1: 197160
Part 2: 145065
